**Import libraries**

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.special import factorial
import h5py
import os
import scipy.io as sio
from google.colab import drive

**Connect dataset from Google Drive**

In [3]:
drive.mount('/content/drive')
path = "drive/MyDrive/CS230_Project/Jenkins_Rstruct_Data"
!ls $path

Mounted at /content/drive
JR_2015-12-04_truncated2.mat  plan_test_data.mat  plan_training_data.mat



## Dataset information

In [4]:
R = sio.loadmat(path+"/JR_2015-12-04_truncated2.mat")["R"][0]
ntrials = len(R)
shape = R[0]['spikeRaster'].todense().shape
fields = R[0].dtype.names

print("There are %d trials in the R-struct" % ntrials)
print("There are %d electrodes with %d millseconds of data \n" % (shape[0], shape[1]))
print("Fields in this dataset include: ")
for field in fields:
    print("-", field)

There are 506 trials in the R-struct
There are 96 electrodes with 901 millseconds of data 

Fields in this dataset include: 
- startDateNum
- startDateStr
- timeTargetOn
- timeTargetAcquire
- timeTargetHeld
- timeTrialEnd
- subject
- counter
- state
- cursorPos
- spikeRaster
- spikeRaster2
- isSuccessful
- trialNum
- timeFirstTargetAcquire
- timeLastTargetAcquire
- trialLength
- target


The fields we care about the most are `target` and `spikeRaster`.

The `target` field holds the coordinates of the reach direction of a given trial.
```python
# returns a tuple indicating the x, y, and z coordinates
# of the target reach direction for trial 0
R[0]['target']
```
The `spikeRaster` field holds a sparse matrix where each row corresponds to an electrode, and each column corresponds to a spike time. We will use the `.todense()` function to convert the row to an array of 1s and 0s where each index is a millisecond indicating whether a neuron fired (1) or not (0).
```python
# returns an array of spikes for the first trial and
# and the first electrode
R[0]['spikeRaster'].todense()[0, :]
# returns whether there was a spike for the first trial and
# and the first electrode during the 10th millisecond
R[0]['spikeRaster'].todense()[0,9]
```

There are 9 possible reach directions.
```python
# Targets sorted in CCW
xy_sorted = np.array([
             [0.0, 0.0],          # 0
             [120.0, 0.0],          # 1
             [84.85, 84.85],        # 2
             [0.0,   120.0],        # 3
             [-84.85,84.85],        # 4
             [-120.0, 0],           # 5
             [-84.85, -84.85],      # 6
             [0.0, -120],           # 7
             [84.85,-84.85]])       # 8
```

## Restructure the data

In [5]:
import tensorflow as tf

In [6]:
num_electrodes = 96
ms = 300
xy_sorted = np.array([
             [0.0, 0.0],
             [120.0, 0.0],
             [84.85, 84.85],
             [0.0, 120.0],
             [-84.85,84.85],
             [-120.0, 0],
             [-84.85, -84.85],
             [0.0, -120],
             [84.85,-84.85]])
# xy_sorted = np.array([
#              [120.0, 0.0],
#              [84.85, 84.85],
#              [0.0, 120.0],
#              [-84.85,84.85],
#              [-120.0, 0],
#              [-84.85, -84.85],
#              [0.0, -120],
#              [84.85,-84.85]])

In [7]:
n_reach_trials = 506 # reaches besides middle
# n_reach_trials = 253 # reaches besides middle
spike_times = np.ndarray(shape=(n_reach_trials, num_electrodes, ms))
targets = np.ndarray(shape=(n_reach_trials, 1), dtype=int)
count = {}
per_reach = {}
total = 0
for n in range(ntrials):
    spike_time = R[n]['spikeRaster'].todense()[:, 200:500]
    target_x = R[n]['target'][0].item()
    target_y = R[n]['target'][1].item()
    for d, dir in enumerate(xy_sorted):
        # col = np.zeros((8,))
        if round(target_x, 2) == round(dir[0], 2) and round(target_y, 2) == round(dir[1], 2):
            if d not in count:
                count[d] = 0
                per_reach[d] = []
            per_reach[d].append(spike_time)
            count[d] += 1
            # col[d] = 1
            targets[total, :] = d
            spike_times[total, :, :] = spike_time
            total += 1
            break
spike_times.shape, targets.shape, total

((506, 96, 300), (506, 1), 506)

In [8]:
count # check distribution of data

{0: 253, 7: 31, 6: 32, 1: 32, 4: 31, 5: 32, 3: 32, 8: 32, 2: 31}

### Evenly Sample From Each Class

In [10]:
import sklearn.model_selection as sk

train_size = .7
# num_train = 173
num_train = 350
# num_train = int(n_reach_trials * .7)
X_train = np.ndarray(shape=(num_train, num_electrodes, ms))
X_test = np.ndarray(shape=(n_reach_trials - num_train, num_electrodes, ms))

Y_train = np.ndarray(shape=(num_train, 1), dtype=int)
Y_test = np.ndarray(shape=(n_reach_trials - num_train, 1), dtype=int)
train_start = 0
test_start = 0
for d in per_reach:
    reach = np.array(per_reach[d])
    target = np.full((reach.shape[0], 1), d)
    reach_x_train, reach_x_test, reach_y_train, reach_y_test = sk.train_test_split(reach, target, train_size=train_size, random_state = 42)
    print(reach_x_train.shape[0], reach_x_test.shape[0])
    X_train[train_start: train_start + reach_x_train.shape[0], :, :] = reach_x_train
    X_test[test_start: test_start + reach_x_test.shape[0], :, :] = reach_x_test
    Y_train[train_start: train_start + reach_y_train.shape[0], :] = reach_y_train
    Y_test[test_start: test_start + reach_y_test.shape[0], :] = reach_y_test

    train_start += reach_x_train.shape[0]
    test_start += reach_x_test.shape[0]


177 76
21 10
22 10
22 10
21 10
22 10
22 10
22 10
21 10


### Convert Data to `tf.data.Dataset`

In [11]:
X_train = tf.cast(X_train, tf.float32)
X_test = tf.cast(X_test, tf.float32)
X_train = tf.data.Dataset.from_tensor_slices(X_train)
X_test = tf.data.Dataset.from_tensor_slices(X_test)
Y_train = tf.data.Dataset.from_tensor_slices(Y_train)
Y_test = tf.data.Dataset.from_tensor_slices(Y_test)

### Apply Map Functions to Data

In [12]:
def one_hot_matrix(label, depth=9):
    """
    Computes the one hot encoding for a single label
    
    Arguments:
        label --  (int) Categorical labels
        depth --  (int) Number of different classes that label can take
    
    Returns:
         one_hot -- tf.Tensor A single-column matrix with the one hot encoding.
    """

    one_hot = tf.reshape(tf.one_hot(label, depth, axis=0), (depth,))
    return one_hot

def normalize(x):
    """
    Transform an image into a tensor of shape (96 * 300, )
    and normalize its components.
    
    Arguments
    image - Tensor.
    
    Returns: 
    result -- Transformed tensor 
    """
    x = tf.reshape(x, [-1,])
    return x

In [13]:
x_train = X_train.map(normalize) # flatten signals
x_test = X_test.map(normalize)

y_train = Y_train.map(one_hot_matrix) # convert classes to one hot
y_test = Y_test.map(one_hot_matrix)

In [14]:
len(y_train), len(x_train)

(350, 350)

In [ ]:
print(next(iter(y_test)))

tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(9,), dtype=float32)


## Create Model

### Helper Functions

In [ ]:
def initialize_parameters():
    """
    Initializes parameters to build a neural network with TensorFlow. The shapes are:
                        W1 : [40, 28800]
                        b1 : [40, 1]
                        W2 : [12, 40]
                        b2 : [12, 1]
                        W3 : [9, 12]
                        b3 : [9, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2, W3, b3
    """
                                
    initializer = tf.keras.initializers.GlorotNormal(seed=1)   

    W1 = tf.Variable(initializer(shape=(40, 28800)))
    b1 = tf.Variable(initializer(shape=(40, 1)))
    W2 = tf.Variable(initializer(shape=(12, 40)))
    b2 = tf.Variable(initializer(shape=(12, 1)))
    W3 = tf.Variable(initializer(shape=(9, 12)))
    b3 = tf.Variable(initializer(shape=(9, 1)))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = tf.math.add(tf.linalg.matmul(W1,  X), b1)
    A1 = tf.keras.activations.relu(Z1)
    Z2 = tf.math.add(tf.linalg.matmul(W2, A1), b2)
    A2 = tf.keras.activations.relu(Z2)
    Z3 = tf.math.add(tf.linalg.matmul(W3, A2), b3)

    
    return Z3

def compute_cost(logits, labels):
    """
    Computes the cost
    
    Arguments:
    logits -- output of forward propagation (output of the last LINEAR unit), of shape (8, num_examples)
    labels -- "true" labels vector, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    logits = tf.transpose(logits)
    labels = tf.transpose(labels)
    # cost = tf.math.reduce_sum(tf.keras.losses.categorical_crossentropy(labels, logits, from_logits=True))
    cost = tf.math.reduce_sum(tf.keras.losses.binary_crossentropy(labels, logits, from_logits=True))
    return cost

### Model

In [ ]:
from keras.optimizers import SGD

def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 4, print_cost = True):
    
    costs = []                                        # To keep track of the cost
    train_acc = []
    test_acc = []
  
    parameters = initialize_parameters()

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    optimizer = tf.keras.optimizers.Adam(learning_rate)
    #f1 score + confusion metrics
    
    # The CategoricalAccuracy will track the accuracy for this multiclass problem
    test_accuracy = tf.keras.metrics.CategoricalAccuracy()
    train_accuracy = tf.keras.metrics.CategoricalAccuracy()
    print(len(X_train), len(Y_train), X_train, Y_train)
    dataset = tf.data.Dataset.zip((X_train, Y_train))
    test_dataset = tf.data.Dataset.zip((X_test, Y_test))
    
    m = dataset.cardinality().numpy()
    
    minibatches = dataset.batch(minibatch_size).prefetch(8)
    test_minibatches = test_dataset.batch(minibatch_size).prefetch(8)
    
    for epoch in range(num_epochs):

        epoch_cost = 0.
        
        train_accuracy.reset_states()
        
        for (minibatch_X, minibatch_Y) in minibatches:
            
            with tf.GradientTape() as tape:
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)
                minibatch_cost = compute_cost(Z3, tf.transpose(minibatch_Y))

            train_accuracy.update_state(minibatch_Y, tf.transpose(Z3))
            
            trainable_variables = [W1, b1, W2, b2, W3, b3]
            grads = tape.gradient(minibatch_cost, trainable_variables)
            optimizer.apply_gradients(zip(grads, trainable_variables))
            epoch_cost += minibatch_cost
        
        epoch_cost /= m

        if print_cost == True and epoch % 10 == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            print("Train accuracy:", train_accuracy.result())
            
            for (minibatch_X, minibatch_Y) in test_minibatches:
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)
                test_accuracy.update_state(minibatch_Y, tf.transpose(Z3))
            print("Test_accuracy:", test_accuracy.result())

            costs.append(epoch_cost)
            train_acc.append(train_accuracy.result())
            test_acc.append(test_accuracy.result())
            test_accuracy.reset_states()


    return parameters, costs, train_acc, test_acc

## Train Model

In [ ]:
parameters, costs, train_acc, test_acc = model(x_train, y_train, x_test, y_test, num_epochs=90)

350 350 <MapDataset element_spec=TensorSpec(shape=(28800,), dtype=tf.float32, name=None)> <MapDataset element_spec=TensorSpec(shape=(9,), dtype=tf.float32, name=None)>
Cost after epoch 0: 0.789389
Train accuracy: tf.Tensor(0.06, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.070512824, shape=(), dtype=float32)
Cost after epoch 10: 0.140474
Train accuracy: tf.Tensor(0.8, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.5, shape=(), dtype=float32)
Cost after epoch 20: 0.044476
Train accuracy: tf.Tensor(0.9771429, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.49358973, shape=(), dtype=float32)
Cost after epoch 30: 0.017080
Train accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.49358973, shape=(), dtype=float32)
Cost after epoch 40: 0.005896
Train accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.49358973, shape=(), dtype=float32)
Cost after epoch 50: 0.003140
Train accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
Test_a